#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- categories: [week26]
- tags: [week26]
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    2. SQLAlchemy db object

## Debugging Flask Object 
![Flask Object](https://github.com/Tirth-Thakkar/APCSP-Blog/blob/master/images/Debugging.png?raw=true)


In [2]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User
    - db.Model inheritance
    - _init_ method
    - ```@property```, ```@<column>.setter```
    - create, read, update, delete methods

In [20]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [21]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Created new uid toby
Created new uid niko
Created new uid lex
Created new uid whit
Created new uid indi
Created new uid raven


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
    2. user.password

In [22]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
    2. user = User(...)
    3. user.dob and try/except
    4. user.create() and try/except

In [23]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Invalid date 6942-25-66 require YYYY-mm-dd, date defaulted to 03-14-2023
Created
 {'id': 7, 'name': 'LOl ', 'uid': 'jerry', 'dob': '03-14-2023', 'age': 0}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [24]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7, 'name': 'LOl ', 'uid': 'jerry', 'dob': '03-14-2023', 'age': 0}]


# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.


- We need routing and need to define attributes and methods for the flask app object
- Creating the class defined the attributes we want in the user
- Inheriting from the database model allows us to use the database methods
- Init method instantiates the object 
- Having setters and getters allow you to change and receive data from the inside of teh an object, can change the initialized value
- Need to have common methods such as create, read, update, delete to have basic functionality and manipulation with the database which help us - interact with the data in the object
- Inside a class we can help solve problems a specific problem with your data
- Inside an object you can see the data being populated 
- Sql Alchemy is an object relational model ORM that works with databases in python which we can use to query things from the database
- Used to have checks such as those for a matching username and password 
- Important to run locally as you don't need all the stuff from others to run your code


In [11]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///computers.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class Computers(db.Model):
    __tablename__ = 'computers'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=True, nullable=False)
    _graphics = db.Column(db.String(255), unique=False, nullable=False)
    _CPU = db.Column(db.String(255), unique=False, nullable=False)
    _price = db.Column(db.Integer, unique=False, nullable=False)
    _RAM = db.Column(db.Integer, unique=False, nullable=False)
    _storage = db.Column(db.Integer, unique=False, nullable=False)
    _OS = db.Column(db.String(255), unique=False, nullable=False)
    _brand = db.Column(db.String(255), unique=False, nullable=False)
    _year = db.Column(db.Integer, unique=False, nullable=False)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, graphics, CPU, price, RAM, storage, OS, brand, year):
        self._name = name    # variables with self prefix become part of the object, 
        self._graphics = graphics
        self._CPU = CPU
        self._price = price
        self._RAM = RAM
        self._storage = storage
        self._OS = OS
        self._brand = brand
        self._year = year

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a name setter method, sets name in object
    @name.setter
    def name(self, name):
        self._name = name

    # a graphics getter method, extracts graphics from object
    @property
    def graphics(self):
        return self._graphics
    
    # a graphics setter method, sets graphics in object
    @graphics.setter
    def graphics(self, graphics):
        self._graphics = graphics

    # a CPU getter method, extracts CPU from object
    @property
    def CPU(self):
        return self._CPU
    
    # a CPU setter method, sets CPU in object
    @CPU.setter
    def CPU(self, CPU):
        self._CPU = CPU
    
    # a price getter method, extracts price from object
    @property
    def price(self):
        return self._price
    
    # a price setter method, sets price in object
    @price.setter
    def price(self, price):
        self._price = price

    # a RAM getter method, extracts RAM from object
    @property
    def RAM(self):
        return self._RAM
    
    # a RAM setter method, sets RAM in object
    @RAM.setter
    def RAM(self, RAM):
        self._RAM = RAM
    
    # a storage getter method, extracts storage from object
    @property
    def storage(self):
        return self._storage
    
    # a storage setter method, sets storage in object
    @storage.setter
    def storage(self, storage):
        self._storage = storage
    
    # a OS getter method, extracts OS from object
    @property
    def OS(self):
        return self._OS
    
    # a OS setter method, sets OS in object
    @OS.setter
    def OS(self, OS):
        self._OS = OS
    
    # a brand getter method, extracts brand from object
    @property
    def brand(self):
        return self._brand
    
    # a brand setter method, sets brand in object
    @brand.setter
    def brand(self, brand):
        self._brand = brand

    # a year getter method, extracts year from object
    @property
    def year(self):
        return self._year
    
    # a year setter method, sets year in object
    @year.setter
    def year(self, year):
        self._year = year
    
    # a method to return a dictionary of the object
    def to_dict(self):
        return {
            "id": self.id,
            "name": self.name,
            "graphics": self.graphics,
            "CPU": self.CPU,
            "price": self.price,
            "RAM": self.RAM,
            "storage": self.storage,
            "OS": self.OS,
            "brand": self.brand,
            "year": self.year,
        }
            

    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None
    def check_price(self):
        if self.price > 1000:
            return True
        else:
            return False
    
    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {          
            "id": self.id,
            "name": self.name,
            "graphics": self.graphics,
            "CPU": self.CPU,
            "price": self.price,
            "RAM": self.RAM,
            "storage": self.storage,
            "OS": self.OS,
            "brand": self.brand,
            "year": self.year,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name, graphics, CPU, price, RAM, storage, OS, brand, year):
        """only updates values with length"""
        
        if len(name) > 0:
            self.name = name
        
        if len(graphics) > 0:
            self.graphics = graphics
        
        if len(CPU) > 0:
            self.CPU = CPU
        
        if price > 0:
            self.price = price
        
        if RAM > 0:
            self.RAM = RAM
        
        if storage > 0:
            self.storage = storage
        
        if len(OS) > 0:
            self.OS = OS
        
        if len(brand) > 0:
            self.brand = brand
        
        if len(year) > 0:
            self.year = year
        
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

In [12]:
"""Database Creation and Testing """


# Builds working data for testing
def initComputers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        c1 = Computers(name='MacBook Pro', graphics='Intel Iris Plus Graphics 645', CPU='2.3GHz 8-core 9th-generation Intel Core i9 processor, Turbo Boost up to 4.8GHz', price=2799, RAM=32, storage=1024, OS='macOS Catalina', brand='Apple', year='2019')
        c2 = Computers(name="Dell XPS 13", graphics="Intel UHD Graphics 620", CPU="2.8GHz 8th Generation Intel Core i7-8565U Processor", price=1299, RAM=16, storage=512, OS="Windows 10 Home", brand="Dell", year="2019")
        c3 = Computers(name='Hp Dragonfly', graphics='Intel UHD Graphics', CPU='Intel i3-1215U', price=1039.00, RAM=16, storage=512, OS='Windows 10 Home', brand='Hp', year='2019')
        c4 = Computers(name='Gigabyte Aero 15 Oled 4k', graphics='3060', CPU='i7 11 gen', price =1440, RAM = 16, storage =512, OS = 'Windows 10',brand = 'Gigabyte', year = '2020')
        c5 = Computers(name='MacBook Air', graphics='Intel UHD', CPU='intel i5', price =1220, RAM = 8, storage =256, OS = 'Mac OS',brand = 'Apple', year = '2020')
        c6 = Computers(name='Lenovo Thinkpad X1 Extreme', graphics='Nvidia Quadro P2000', CPU='Intel Core i7-9750H', price =2000, RAM = 16, storage =512, OS = 'Windows 10',brand = 'Lenovo', year = '2020')
        c7 = Computers(name="Acer Nitro 5", graphics="Nvidia GeForce RTX 3070ti", CPU="Intel Core i7-12700H", price=3, RAM=32, storage=2048, OS="Windows 11 Home", brand="Acer", year="2022")

        Comps = [c1, c2, c3, c4, c5, c6, c7]

        """Builds sample user/note(s) data"""
        for Computer in Comps:
            try:
                '''add user to table'''
                object = Computer.create()
                print(f"Created new record {Computer.name}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {Computer.name}, or error.")
                
initComputers()

# SQLAlchemy extracts single user from database matching User ID
def name(name):
    with app.app_context():
        computer = Computers.query.filter_by(_name=name).first()
    return computer # returns user object


Created new record MacBook Pro
Created new record Dell XPS 13
Created new record Hp Dragonfly
Created new record Gigabyte Aero 15 Oled 4k
Created new record MacBook Air
Created new record Lenovo Thinkpad X1 Extreme
Created new record Acer Nitro 5


In [13]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    name = input("Enter the name of your computer:")
    graphics = input("Enter the graphics of your computer:")
    CPU = input("Enter the CPU of your computer:")
    price = input("Enter the price of your computer (integer):")
    price = int(price)
    RAM = input("Enter the RAM of your computer (integer):")
    RAM = int(RAM)
    storage = input("Enter the storage of your computer (integer):")
    storage = int(storage)
    OS = input("Enter the OS of your computer:")
    brand = input("Enter the brand of your computer:")
    year = input("Enter the year of your computer:")

        # Initialize User object before date
    comp = Computers(name=name, graphics=graphics, CPU=CPU, price=price, RAM=RAM, storage=storage, OS=OS, brand=brand, year=year)
       
    # write object to database
    with app.app_context():
        try:
            object = comp.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error occurred, or duplicate data.")
        
create()

Created
 {'id': 11, 'name': 'apple 2e', 'graphics': 'apple special', 'CPU': 'old', 'price': 1231342341, 'RAM': 12, 'storage': 12, 'OS': 'MacOs', 'brand': 'Apple', 'year': 1000}


In [19]:
# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = Computers.query.all()
    json_ready = [computer.read() for computer in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 2,
  'name': 'Dell XPS 13',
  'graphics': 'Intel UHD Graphics 620',
  'CPU': '2.8GHz 8th Generation Intel Core i7-8565U Processor',
  'price': 1299,
  'RAM': 16,
  'storage': 512,
  'OS': 'Windows 10 Home',
  'brand': 'Dell',
  'year': 2019},
 {'id': 3,
  'name': 'Hp Dragonfly',
  'graphics': 'Intel UHD Graphics',
  'CPU': 'Intel i3-1215U',
  'price': 1039,
  'RAM': 16,
  'storage': 512,
  'OS': 'Windows 10 Home',
  'brand': 'Hp',
  'year': 2019},
 {'id': 4,
  'name': 'Gigabyte Aero 15 Oled 4k',
  'graphics': '3060',
  'CPU': 'i7 11 gen',
  'price': 1440,
  'RAM': 16,
  'storage': 512,
  'OS': 'Windows 10',
  'brand': 'Gigabyte',
  'year': 2020},
 {'id': 5,
  'name': 'MacBook Air',
  'graphics': 'Intel UHD',
  'CPU': 'intel i5',
  'price': 1220,
  'RAM': 8,
  'storage': 256,
  'OS': 'Mac OS',
  'brand': 'Apple',
  'year': 2020},
 {'id': 6,
  'name': 'Lenovo Thinkpad X1 Extreme',
  'graphics': 'Nvidia Quadro P2000',
  'CPU': 'Intel Core i7-9750H',
  'price': 2000,
  'RAM': 16,
 

In [21]:
def delete():
    # Gets the ID
    DComp = input("Enter the computer id to be deleted:")
    with app.app_context():
        # Gets the user through the ID
        ComputerDelete = Computers.query.get(DComp)
        if ComputerDelete:
            # Deletes the user according to its ID number
            ComputerDelete.delete()
            print(f"Computer with id {DComp} deleted")
        else:
            # Error message if delete fails
            print(f"Computer  with id {DComp} not found")
    
delete()

Computer with id 2 deleted


/tmp/ipykernel_2076/934772255.py:6: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  ComputerDelete = Computers.query.get(DComp)


In [14]:
def update():
    name = input("Enter the name of your computer:")
    graphics = input("Enter the graphics of your computer:")
    CPU = input("Enter the CPU of your computer:")
    price = int(input("Enter the price of your computer (integer):"))
    RAM = int(input("Enter the RAM of your computer (integer):"))
    storage = int(input("Enter the storage of your computer (integer):"))
    OS = input("Enter the OS of your computer:")
    brand = input("Enter the brand of your computer:")
    year = input("Enter the year of your computer:")

    if len(name) <= 0:
        return {"error": "Name is required"}
    if len(graphics) <= 0:
        return {"error": "Graphics is required"}
    if len(CPU) <= 0:
        return {"error": "CPU is required"}
    if price <= 0:
        return {"error": "Price is required"}
    if RAM <= 0:
        return {"error": "RAM is required"}
    if storage <= 0:
        return {"error": "Storage is required"}
    if len(OS) <= 0:
        return {"error": "OS is required"}
    if len(brand) <= 0:
        return {"error": "Brand is required"}
    if len(year) <= 0:
        return {"error": "Year is required"}
    
    with app.app_context():
        # Gets the user through the ID
        ComputerUpdate = Computers.query.filter_by(_name=name).first()
        if ComputerUpdate:
            ComputerUpdate.update(name=name, graphics=graphics, CPU=CPU, price=price, RAM=RAM, storage=storage, OS=OS, brand=brand, year=year) 

            print(f"Computer with id {name} updated")
        else:
            # Error message if delete fails
            print(f"Computer  with id {name} not found")

update()

Computer with id apple 2e updated


In [23]:
# Menu, to run other cells from one control point
def menu():
    operation = input("Enter: (C)reate (R)ead (U)pdate or (D)elete")
    if operation.lower() == 'c':
        create()
    elif operation.lower() == 'r':
        read()
    elif operation.lower() == 'u':
        update()
    elif operation.lower() == 'd':
        delete()
    elif len(operation)==0: # Escape Key
        return
    else:
        print("Please enter c, r, u, or d") 
    menu() # recursion, repeat menu
        
try:
    menu() # start menu
except:
    print("Perform Jupyter 'Run All' prior to starting menu")

Computer with id apple 2e updated
